<a href="https://colab.research.google.com/github/rosacarla/modelo-abalone-lgbmregressor/blob/main/abalone_somativa2_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

># ANÉIS DE ABALONE: TREINAMENTO DE MODELO PREDITIVO
>
>CURSO: Tecnologia em Inteligência Artificial Aplicada  
>DISCIPLINA: Técnicas de Machine Learning  
>PROFESSOR: Wellington Monteiro  
>DATA: 27/11/2023  
>
> <img src='https://i.postimg.cc/bJXP6cMq/abalone.png' height=256 width=960>

## SOBRE ESTE NOTEBOOK

 - Entradas: conjunto de dados ```abalone.csv```, composto por 4177 linhas e 9 colunas, com dados de um tipo de molusco marinho (por exemplo, sexo, comprimento, diâmetro, peso, dentre outros), a partir dos quais há como prever a quantidade de anéis (variável “Rings”) do animal; a idade dessa espécie pode ser determinada pela contagem dos seus anéis e, para fazer isto, é necessário cortar a concha do animal e contar os anéis com auxílio de um microscópio.
 - Saídas: o resultado gera previsões para a variável Rings ou quantidade de anéis do abalone.
 - Período: são dados sobre 8 características atribuídas ao molusco abalone; não se localizaou a data de coleta dos dados; sabe-se que o conjunto de dados foi doado em 30/11/1995 ao repositório de machine learning da Universidade da Califórnia Irvine.
 - Objetivo: realizar o treinamento de modelo preditivo com um algoritmo de aprendizagem supervisionada em sequências diferentes de passos e, desse modo, responder à parte 2 da proposta de avaliação somativa da disciplina Técnicas de Machine Learning.
 - Autoria: Carla Edila Silveira

## 1a. MODIFICAÇÃO DO NOTEBOOK

In [ ]:
# Importa as bibliotecas
import pandas as pd # utilizado para manipular o dataset
import matplotlib.pyplot as plt # módulo para criar gráficos e visualizações
import numpy as np  # utilizado para realizar operações numéricas
import seaborn as sns  # para visualizar dados em gráficos estatísticos
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, RobustScaler  # utilizado para converter colunas do tipo texto para representação binária
from sklearn.model_selection import train_test_split # para dividir dados de treinamento e teste
from lightgbm import LGBMRegressor # modelo de LGBM Regressor
from sklearn.pipeline import Pipeline # utilizado para criar pipelines
from sklearn.compose import TransformedTargetRegressor # cria modelo de regressão onde a variável alvo é transformada antes de ajustar o modelo
from sklearn.preprocessing import QuantileTransformer # Transforma distribuição dos valores em distribuição uniforme ou normal
from sklearn.compose import ColumnTransformer  # aplica diferentes transformações em diferentes colunas do conjunto de dados
from sklearn.metrics import mean_squared_error, mean_absolute_error # metricas para avaliação do desempenho de modelos de regressão
from sklearn.model_selection import GridSearchCV # faz busca em grade para encontrar melhores hiperparametros de um modelo

from sklearn import set_config # utilizado para exibir visualmente os passos do pipeline
set_config(display='diagram') # força para que passos do pipeline sejam exibidos visualmente

# Importa módulo warnings que manipula mensagems de alerta sobre situações que não são erros
import warnings
warnings.filterwarnings('ignore') # Ignora avisos exibidos durante a execução do código

### CARREGAMENTO DOS DADOS

In [ ]:
# Carregamento e leitura do  dataset
df = pd.read_csv('abalone.csv')

# Visualização inicial do dataset
df

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


DICIONÁRIO DE DADOS  

| VARIÁVEL | DESCRIÇÃO |
| ------------ | ------------ |
|**Sex**| sexo, subdividido em três tipos: M (masculino), F (feminino), I (infantil)|
|**Length**| comprimento, a medida mais longa da concha|
|**Diameter**| diâmetro, é perpendicular ao comprimento|
|**Height**| altura, inclui a carne com a casca|
|**WholeWeight**| peso total, o peso do abalone inteiro|
|**ShuckedWeight**| peso sem casca, o peso da carne|
|**VisceraWeight**| peso das vísceras, o peso intestinal após o sangramento|
|**ShellWeight**| peso da casca depois de seca|
|**Rings**| anéis, adicionar 1,5 aos anéis dá a idade do abalone|

In [ ]:
# Mostra linhas de Height com valor 0

#df[df['Height']==0]

In [ ]:
# Ignora dados em que Height = 0

#df = df[df['Height']>0]

In [ ]:
# Mostra composição do dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Sex            4177 non-null   object 
 1   Length         4177 non-null   float64
 2   Diameter       4177 non-null   float64
 3   Height         4177 non-null   float64
 4   WholeWeight    4177 non-null   float64
 5   ShuckedWeight  4177 non-null   float64
 6   VisceraWeight  4177 non-null   float64
 7   ShellWeight    4177 non-null   float64
 8   Rings          4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


OBSERVAÇÃO: O dataframe está composto por 4.177 linhas, 9 colunas (sendo 1 categórica object, 7 numéricas decimais float64, 1 numérica inteira int64).

### DIVISÃO DO DATASET

In [ ]:
# Divide o conjunto de dados em treino e teste (75% - 25%)
# X contém as variáveis preditoras (features), exceto Rings
# y contem a variável alvo Rings

X_train, X_test, y_train, y_test = train_test_split(df.drop('Rings', axis=1),
                                                    df['Rings'],
                                                    test_size=0.25,
                                                    random_state=42)

### PREPARAÇÃO DOS DADOS

In [ ]:
# Transforma variável categórica Sex em variável numérica
# Cria uma instância do OrdinalEncoder
encoder = OrdinalEncoder()

# Ajusta e transforma os dados de treino
X_train[['Sex']] = encoder.fit_transform(X_train[['Sex']])

# Transforma os dados de teste
X_test[['Sex']] = encoder.transform(X_test[['Sex']])

In [ ]:
# Visualiza X_train após transformação
X_train.head(3)

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight
3823,0.0,0.615,0.455,0.135,1.059,0.4735,0.2630,0.274
3956,0.0,0.515,0.395,0.140,0.686,0.2810,0.1255,0.220
3623,2.0,0.660,0.530,0.175,1.583,0.7395,0.3505,0.405


In [ ]:
# Visualiza X_test após transformação
X_test.head(3)

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight
866,2.0,0.605,0.455,0.160,1.1035,0.421,0.3015,0.325
1483,2.0,0.590,0.440,0.150,0.8725,0.387,0.2150,0.245
599,0.0,0.560,0.445,0.195,0.9810,0.305,0.2245,0.335


In [ ]:
# Aplica QuantileTransformer que mapeia os dados para uma distribuição uniforme ou normal
# Ajuda a reduzir a sensibilidade a outliers e melhorar o desempenho do modelo

# Aplica o QuantileTransformer às variaveis preditoras
quantile_transformer = QuantileTransformer()
X_train_transformed = quantile_transformer.fit_transform(X_train)
X_test_transformed = quantile_transformer.transform(X_test)

### DEFINIÇÃO DO PROBLEMA
Considerando a estrutura do dataset e o contexto descrito inicialmente, define-se que o problema pode ser tratado como um **problema de regressão**, pois há intenção de **prever um valor numérico (a quantidade de anéis)** a partir de várias características dos abalones.  
Dado isso, a aplicação de um algoritmo de regressão seria adequada para construir um modelo que possa prever a quantidade de anéis com base nas características fornecidas no conjunto de dados. As opções de algoritmos são regressão linear, regressão de árvore de decisão, SVM para regressão, random forest, entre outros, para lidar com a tarefa de previsão.

### TREINAMENTO DO MODELO LGBM REGRESSOR

In [ ]:
# Inicializa e treina modelo LGBMRegressor
lgbm_model = LGBMRegressor()

# Aplica o QuantileTransformer à variável alvo durante o treinamento
# TransformedTargetRegressor opcionalmente tem transformação inversa aplicada às previsões
quantile_transformer = QuantileTransformer()
transformed_target_regressor = TransformedTargetRegressor(regressor=lgbm_model, transformer=quantile_transformer)

In [ ]:
# Treina o modelo com o TransformedTargetRegressor
transformed_target_regressor.fit(X_train, y_train)

# lgbm_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000424 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1293
[LightGBM] [Info] Number of data points in the train set: 3132, number of used features: 8
[LightGBM] [Info] Start training from score 0.500119


TransformedTargetRegressor(regressor=LGBMRegressor(),
                           transformer=QuantileTransformer())

In [ ]:
# Faz previsões no conjunto de teste - LGBM
lgbm_pred_test = transformed_target_regressor.predict(X_test)
#lgbm_pred_test = lgbm_model.predict(X_test)
lgbm_pred_test

array([11.,  9., 13., ..., 11., 11.,  9.])

### VISUALIZAÇÃO DAS PREDIÇÕES NA BASE DE TESTE

In [ ]:
# Visualização da predições do LGBM Regressor
# Compara valores de teste e previstos
results_regression = pd.DataFrame(y_test)
results_regression['Predicao_LGBM'] = lgbm_pred_test

results_regression.head()

,Rings,Predicao_LGBM
866,9,11.0
1483,8,9.0
599,16,13.0
1702,9,10.0
670,14,10.0


## 2a. MODIFICAÇÃO DO NOTEBOOK COM PIPELINE

### CARREGAMENTO DOS DADOS

In [ ]:
# Cria cópia do dataset para 2a. modificação do notebook
df2 = df.copy()

# Visualiza primeiras linhas do dataset copiado
df2.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


### DIVISÃO DO DATASET

In [ ]:
# Divide o conjunto de dados em treino e teste (75% - 25%)
# X contém as variáveis preditoras (features), exceto Rings
# y contpem a variável alvo Rings

X_train, X_test, y_train, y_test = train_test_split(df2.drop('Rings', axis=1),
                                                    df2['Rings'],
                                                    test_size=0.25,
                                                    random_state=42)

### PIPELINE COM PREPARAÇÃO DOS DADOS E TREINAMENTO

In [ ]:
# Cria pipeline com o LGBMRegressor, QuantileTransformer e OrdinalEncoder
pipe_lgbm = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('numeric_quantile', QuantileTransformer(), X_train.select_dtypes(include=['number']).columns),  # Aplica o QuantileTransformer a todas as colunas numéricas
            ('categorical_encoder', OrdinalEncoder(), ['Sex'])  # Aplica o OrdinalEncoder à variável categórica 'Sex'
        ],
        remainder='passthrough'
    )),
    ('modelo', LGBMRegressor())
])

# Treina o modelo usando a pipeline
pipe_lgbm.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000465 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1292
[LightGBM] [Info] Number of data points in the train set: 3132, number of used features: 8
[LightGBM] [Info] Start training from score 9.964879


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric_quantile',
                                                  QuantileTransformer(),
                                                  Index(['Length', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight',
       'VisceraWeight', 'ShellWeight'],
      dtype='object')),
                                                 ('categorical_encoder',
                                                  OrdinalEncoder(),
                                                  ['Sex'])])),
                ('modelo', LGBMRegressor())])

In [ ]:
# Mostra passos da pipeline
display(f'Passos do Pipeline:')
pipe_lgbm

'Passos do Pipeline:'

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric_quantile',
                                                  QuantileTransformer(),
                                                  Index(['Length', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight',
       'VisceraWeight', 'ShellWeight'],
      dtype='object')),
                                                 ('categorical_encoder',
                                                  OrdinalEncoder(),
                                                  ['Sex'])])),
                ('modelo', LGBMRegressor())])

In [ ]:
# Faz previsões no conjunto de teste LGBM
lgbm_pred_test = pipe_lgbm.predict(X_test)
lgbm_pred_test

array([12.08467582,  9.64225372, 14.69359588, ..., 11.72593311,
       12.08252064,  9.12358313])

In [ ]:
# Visualiza predições da LGBM Regressor - compara valores de teste e previstos
results_regression = pd.DataFrame(y_test)
results_regression['Predicao_LGBM'] = lgbm_pred_test
results_regression.head()

,Rings,Predicao_LGBM
866,9,12.084676
1483,8,9.642254
599,16,14.693596
1702,9,10.883314
670,14,11.265107


### AVALIAÇÃO DO MODELO LBGM REGRESSOR

In [ ]:
#  Métricas de avaliação do modelo de LGBM Regressor - 2a. modificação
# Exibe métricas dos resultados do pipeline

# Erro absoluto médio - MAE
# Calcula a média das diferenças absolutas entre valores previstos e valores reais dos dados de teste
#display(f"Resultado do MAE: {mean_absolute_error(y_test, rlin_pred_test)}")
#display(f"Resultado do MAE: {mean_absolute_error(y_test, xgb_pred_test)}")
#display(f"Resultado do MAE: {mean_absolute_error(y_test, rf_pred_test)}")
display(f"Resultado do MAE: {mean_absolute_error(y_test, lgbm_pred_test)}") # melhorou com QuantileTransformer

# Erro Quadrático Médio - MSE
# Calcula a média dos quadrados dos erros entre as previsões e os valores reais dos dados de teste
#display(f"Resultado do MSE: {mean_squared_error(y_test, rlin_pred_test)}")
#display(f"Resultado do MSE: {mean_squared_error(y_test, xgb_pred_test)}")
#display(f"Resultado do MSE: {mean_squared_error(y_test, rf_pred_test)}")
display(f"Resultado do MSE: {mean_squared_error(y_test, lgbm_pred_test)}")

# Raiz do Erro Quadrático Médio - RMSE
# Calcula a raiz quadrada da média dos quadrados das diferenças entre os valores reais e os preditos
#display(f"Resultado do RMSE: {mean_squared_error(y_test, rlin_pred_test, squared=False)}")
#display(f"Resultado do RMSE: {mean_squared_error(y_test, xgb_pred_test, squared=False)}")
#display(f"Resultado do RMSE: {mean_squared_error(y_test, rf_pred_test, squared=False)}")
display(f"Resultado do RMSE: {mean_squared_error(y_test, lgbm_pred_test, squared=False)}")

#xgb com QuantileTransformer
# Resultado do MAE: 1.6221016895257685
# Resultado do MSE: 5.387380472552362
# Resultado do RMSE: 2.3210731295140965

# rf com QuantileTransformer
# Resultado do MAE: 1.5659425837320573
# Resultado do MSE: 4.835143444976077
# Resultado do RMSE: 2.19889596047109

# rf sem QuantileTransformer
# Resultado do MAE: 1.5606985645933014
# Resultado do MSE: 4.815378277511961
# Resultado do RMSE: 2.1943970191175435

'Resultado do MAE: 1.5178962207957012'

'Resultado do MSE: 4.743160857167622'

'Resultado do RMSE: 2.177879899619725'

OBSERVAÇÃO: Quanto menor é o valor do MAE, melhor é o desempenho do modelo, assim um MAE próximo de 0 significa que as previsões do modelo estão mais próximas dos valores reais. RMSE está na mesma escala que a variável de resposta e, quanto menor é, melhor o modelo se ajusta aos dados observados.

In [ ]:
# AJUSTE DE HIPERPARÂMETROS DO MODELO LGBM REGRESSOR
# Define hiperparâmetros para ajustar
param_grid = {
    'modelo__n_estimators': [50, 100, 200],  # Número de árvores no modelo
    'modelo__learning_rate': [0.01, 0.05, 0.1],  # Taxa de aprendizado
    'modelo__max_depth': [3, 5, 7],  # Profundidade máxima das árvores
}

In [ ]:
# Cria pipeline com o LGBMRegressor, QuantileTransformer e OrdinalEncoder
pipe_lgbm = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('numeric_quantile', QuantileTransformer(), X_train.select_dtypes(include=['number']).columns),
            ('categorical_encoder', OrdinalEncoder(), ['Sex'])
        ],
        remainder='passthrough'
    )),
    ('modelo', LGBMRegressor())
])

In [ ]:
# Mostra passos da pipeline
display(f'Passos do Pipeline:')
pipe_lgbm

'Passos do Pipeline:'

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric_quantile',
                                                  QuantileTransformer(),
                                                  Index(['Length', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight',
       'VisceraWeight', 'ShellWeight'],
      dtype='object')),
                                                 ('categorical_encoder',
                                                  OrdinalEncoder(),
                                                  ['Sex'])])),
                ('modelo', LGBMRegressor())])

In [ ]:
# Cria o objeto GridSearchCV
grid_search = GridSearchCV(pipe_lgbm, param_grid, cv=5, scoring='neg_mean_squared_error')

# Treina o modelo usando a busca em grade
grid_search.fit(X_train, y_train)

A saída de streaming foi truncada nas últimas 5000 linhas.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('numeric_quantile',
                                                                         QuantileTransformer(),
                                                                         Index(['Length', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight',
       'VisceraWeight', 'ShellWeight'],
      dtype='object')),
                                                                        ('categorical_encoder',
                                                                         OrdinalEncoder(),
                                                                         ['Sex'])])),
                                       ('modelo', LGBMRegressor())]),
             param_grid={'modelo__learning_rate': [0.01, 0.05, 0.1],
                         'modelo__max_depth': [3, 5, 7],
                         'modelo__n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error')

In [ ]:
# Exibe os melhores hiperparâmetros encontrados
print("Melhores Hiperparâmetros:")
print(grid_search.best_params_)


# SAIDAS
# Melhores Hiperparâmetros:
# {'modelo__learning_rate': 0.05, 'modelo__max_depth': 7, 'modelo__n_estimators': 100}
# [LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
# Mean Squared Error: 4.579627724914982

Melhores Hiperparâmetros:
{'modelo__learning_rate': 0.05, 'modelo__max_depth': 7, 'modelo__n_estimators': 100}


In [ ]:
# Faz previsões no conjunto de teste usando o melhor modelo
lgbm_pred_test = grid_search.predict(X_test)
lgbm_pred_test

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


array([12.33063126,  9.86428501, 14.28507849, ..., 11.52325898,
       11.99062314,  9.15311407])

In [ ]:
# Visualiza predições da LGBM Regressor com melhores parametros - compara valores de teste e previstos
results_regression = pd.DataFrame(y_test)
results_regression['Predicao_LGBM'] = lgbm_pred_test
results_regression.head()

,Rings,Predicao_LGBM
866,9,12.330631
1483,8,9.864285
599,16,14.285078
1702,9,10.963956
670,14,11.107366


In [ ]:
# Métricas de avaliação da Regressão Linear após ajuste de hiperparâmetros
# Erro absoluto médio - MAE
display(f"Resultado do MAE: {mean_absolute_error(y_test, lgbm_pred_test)}")

# Erro Quadrático Médio - MSE
display(f"Resultado do MSE: {mean_squared_error(y_test, lgbm_pred_test)}")

# Raiz do Erro Quadrático Médio - RMSE
display(f"Resultado do RMSE: {mean_squared_error(y_test, lgbm_pred_test, squared=False)}")

'Resultado do MAE: 1.4961228540146514'

'Resultado do MSE: 4.579627724914982'

'Resultado do RMSE: 2.14000647777407'

In [ ]:
# Mostra dados estatísticos dos dados de teste
# Gerado para auxiliar na análise das métricas do modelo
y_test.describe()

count    1045.000000
mean        9.840191
std         3.208930
min         3.000000
25%         8.000000
50%         9.000000
75%        11.000000
max        23.000000
Name: Rings, dtype: float64

CONSIDERAÇÕES FINAIS:   
A princípio, o modelo LGBM Regressor não teve desempenho melhor do que no treinamento realizado com a Regressão Linear na primeira parte da atividade somativa (demonstrado no notebook abalone-somativa1). Os resultados da RL obtiveram métricas com valores de 0, conforme segue:

| MÉTRICA    | RESULTADO  |
|:---------- |:---------- |
|MAE| 0.15733573292080075 |
|MSE| 0.05091973704737251 |

Como houve o pré-processamento antes da divisão dos dados de treino e teste, não se pode considerar o resultado como satisfatório porque a técnica não condiz com a prática habitual.

Nesta segunda parte da atividade somativa, houve duas modificações no notebook. Primeiro, dividiram-se os dados para treino e teste, houve a preparação dos dados, seguida do treinamento. Na segunda alteração, após divisão dos dados, incluiu-se um pipeline contendo a preparação dos dados e treinamento do modelo. Ao finalizar as modificações, as métricas do LGBM apresentaram resultados um pouco melhores dos que os obtidos na tentativa com a Regressão Linear em pipeline com PCA. Seguem abaixo os resultados para comparação do quanto se conseguiu melhorar.

###### REGRESSÃO LINEAR - PIPELINE COM PCA
| MÉTRICA   | RESULTADO  |
|:----------|:---------- |
|MAE|  1.963593414973981 |
|MSE| 6.9252491605312745 |
|RMSE| 2.631586814173394 |

###### LGBM REGRESSOR - PIPELINE COM QUANTILETRANFORMER
| MÉTRICA   | RESULTADO  |
|:----------|:---------- |
|MAE| 1.5178962207957012 |
|MSE|  4.743160857167622 |
|RMSE| 2.177879899619725 |


Para interpretar a avaliação, recorreu-se à comparação das métricas com as estatísticas dos dados de teste, como Média e Desvio Padrão. O MAE indica que, em média, as previsões desviam em torno de 1.51 unidades dos valores reais. O RMSE indica que, em média, os erros do modelo atingem 2.17 unidades. Os valores de MAE e RMSE, por serem menores que a Média dos dados de teste (9.84), indicam que o modelo está, em média, fazendo previsões próximas aos valores reais. Em relação ao Desvio Padrão dos dados de teste (cerca de 3.21), MAE e RMSE também são menores. Isto significa que as previsões do modelo estão, em média, dentro da faixa de variabilidade dos dados observados.  

Mesmo com o ajuste de hiperparâmetros, o modelo de LGBM Regressor obteve pouca melhoria no seu desempenho. As métricas ficaram assim:

| MÉTRICA   | RESULTADO  |
|:----------|:---------- |
|MAE| 1.4961228540146514 |
|MSE| 4.579627724914982  |
|RMSE| 2.14000647777407  |

Por fim, conclui-se que é necessário pesquisar e testar outras técnicas de machine learning para aprimorar a qualidade deste modelo, pois mesmo depois de treinamentos realizados com outros algoritmos de regressão os resultados apresentaram melhora pouco significativa.
